In [1]:
import torch
import torchvision.datasets as datasets 
import torchvision.transforms as transforms
import torch.nn as nn
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
_ = torch.manual_seed(0)

In [3]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=10, shuffle=True)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=10, shuffle=True)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9912422/9912422 [00:25<00:00, 390376.05it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28881/28881 [00:00<00:00, 92990.53it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1648877/1648877 [00:02<00:00, 687119.43it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4542/4542 [00:00<00:00, 1090034.26it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [7]:


class Richboy(nn.Module):
    def __init__(self, hidden_size_1, hidden_size_2):
        super(Richboy, self).__init__()
        self.linear1 = nn.Linear(28*28, hidden_size_1)
        self.linear2 = nn.Linear(hidden_size_1, hidden_size_2)
        self.linear3 = nn.Linear(hidden_size_2, 10)
        self.relu = nn.ReLU()
    
    def forward(self, img):
        x = img.view(-1, 28*28)
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.linear3(x)
        return x

# Example usage:
net = Richboy(hidden_size_1=128, hidden_size_2=64)

In [9]:
def train(train_loader,net,epochs=5,total_iterations_limit=None):
     cross_el = nn.CrossEntropyLoss()
     optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
     total_iterations = 0
     for epoch in range(epochs):
          net.train()
          loss_sum = 0
          num_iterations = 0
          data_iterator=tqdm(train_loader, desc=f'Epoch {epoch+1}')
          if total_iterations_limit is not None:
            data_iterator.total = total_iterations_limit
          for data in data_iterator:
              num_iterations += 1
              total_iterations += 1
              x, y = data
              optimizer.zero_grad()
              output = net(x.view(-1, 28*28))
              loss = cross_el(output, y)
              loss_sum += loss.item()
              avg_loss = loss_sum / num_iterations
              data_iterator.set_postfix(loss=avg_loss)
              loss.backward()
              optimizer.step()

train(train_loader, net, epochs=1)



Epoch 1: 100%|██████████| 6000/6000 [00:14<00:00, 423.25it/s, loss=0.222]


In [10]:
original_weights = {}
for name, param in net.named_parameters():
    original_weights[name] = param.clone().detach()

In [13]:
def test():
    correct = 0
    total = 0

    wrong_counts = [0 for i in range(10)]

    with torch.no_grad():
        for data in tqdm(test_loader, desc='Testing'):
            x, y = data
            output = net(x.view(-1, 784))
            for idx, i in enumerate(output):
                if torch.argmax(i) == y[idx]:
                    correct +=1
                else:
                    wrong_counts[y[idx]] +=1
                total +=1
    print(f'Accuracy: {round(correct/total, 3)}')
    for i in range(len(wrong_counts)):
        print(f'wrong counts for the digit {i}: {wrong_counts[i]}')

test()

Testing: 100%|██████████| 1000/1000 [00:00<00:00, 1633.57it/s]

Accuracy: 0.966
wrong counts for the digit 0: 11
wrong counts for the digit 1: 17
wrong counts for the digit 2: 41
wrong counts for the digit 3: 57
wrong counts for the digit 4: 62
wrong counts for the digit 5: 30
wrong counts for the digit 6: 27
wrong counts for the digit 7: 25
wrong counts for the digit 8: 26
wrong counts for the digit 9: 42


In [14]:
total_parameters_original = 0
for index, layer in enumerate([net.linear1, net.linear2, net.linear3]):
    total_parameters_original += layer.weight.nelement() + layer.bias.nelement()
    print(f'Layer {index+1}: W: {layer.weight.shape} + B: {layer.bias.shape}')
print(f'Total number of parameters: {total_parameters_original:,}')

Layer 1: W: torch.Size([128, 784]) + B: torch.Size([128])
Layer 2: W: torch.Size([64, 128]) + B: torch.Size([64])
Layer 3: W: torch.Size([10, 64]) + B: torch.Size([10])
Total number of parameters: 109,386


In [ ]:
class LoRAParametrization(nn.Module):
    def __init__(self, features_in, features_out, rank=1, alpha=1):
        super().__init__()
        self.lora_A = nn.Parameter(torch.zeros((rank,features_out)))
        self.lora_B = nn.Parameter(torch.zeros((features_in, rank)))
        nn.init.normal_(self.lora_A, mean=0, std=1)
        self.scale = alpha / rank
        self.enabled = True

    def forward(self, original_weights):
        if self.enabled:
            return original_weights + torch.matmul(self.lora_B, self.lora_A).view(original_weights.shape) * self.scale
        else:
            return original_weights

In [ ]:
import torch.nn.utils.parametrize as parametrize

def linear_layer_parameterization(layer, device, rank=1, lora_alpha=1):
    
    features_in, features_out = layer.weight.shape
    return LoRAParametrization(
        features_in, features_out, rank=rank, alpha=lora_alpha
    )

parametrize.register_parametrization(
    net.linear1, "weight", linear_layer_parameterization(net.linear1)
)
parametrize.register_parametrization(
    net.linear2, "weight", linear_layer_parameterization(net.linear2)
)
parametrize.register_parametrization(
    net.linear3, "weight", linear_layer_parameterization(net.linear3)
)


def enable_disable_lora(enabled=True):
    for layer in [net.linear1, net.linear2, net.linear3]:
        layer.parametrizations["weight"][0].enabled = enabled